In [5]:
import os
import pandas as pd

In [6]:
# Read in any network_summary_detailed file in the working directory

In [7]:
# input_dir = sys.argv[1]
# output_dir = sys.argv[2]
input_dir = r'C:\Users\Brice\surveys\surveys\net_summaries'
output_dir = r'C:\Users\Brice\surveys\surveys\net_summaries'

In [8]:
# look up psrc time of day 
tod_list = ['5to6','6to7','7to8','8to9','9to10','10to14','14to15','16to17','17to18','18to20']

tod_lookup = {  0:'20to5',
                1:'20to5',
                2:'20to5',
                3:'20to5',
                4:'20to5',
                5:'5to6',
                6:'6to7',
                7:'7to8',
                8:'8to9',
                9:'9to10',
                10:'10to14',
                11:'10to14',
                12:'10to14',
                13:'10to14',
                14:'14to15',
                15:'15to16',
                16:'16to17',
                17:'17to18',
                18:'18to20',
                19:'18to20',
                20:'18to20',
                21:'20to5',
                22:'20to5',
                23:'20to5' }

In [9]:
def main():
    output_csv_list = ['transit_boardings','traffic_counts','net_summary']

    overwrite = True

    if overwrite:
        for fname in output_csv_list:
            if os.path.isfile(os.path.join(output_dir,fname+'.csv')):
                os.remove(os.path.join(output_dir,fname+'.csv'))

    for fname in os.listdir(input_dir):
        if fname.endswith('.xlsx'):
            net_file = os.path.join(input_dir,fname)

            print 'processing ' + fname

            transit_summary(net_file, fname)
            traffic_counts(net_file, fname)
            net_summary(net_file, fname)

In [10]:
def write_csv(df, fname):
    '''
    Write dataframe to file; append existing file
    '''

    if not os.path.isfile(os.path.join(output_dir,fname)):
        df.to_csv(os.path.join(output_dir,fname), index=False)
    else: # append without writing the header
        df.to_csv(os.path.join(output_dir,fname), mode ='a', header=False, index=False)

In [11]:
def transit_summary(net_file, fname):
    
    transit_df = pd.read_excel(net_file, sheetname='Transit Summaries')
    transit_df.index = transit_df['route_code']
    # fname = 'transit_boardings.csv'

    # Add model results
    dict_result = {}
    for field in ['board','time']:
        df = pd.DataFrame(transit_df[[tod+'_'+ field for tod in tod_list]].stack())
        df.rename(columns={0:field}, inplace=True)
        df['tod'] = [i.split('_')[0] for i in df.index.get_level_values(1)]
        df['route_id'] = df.index.get_level_values(0)
        df.reset_index(inplace=True, drop=True)

        dict_result[field] = df

    # Only keep the boardings for now - observed time data is not available at the route level
    df = dict_result['board'].groupby(['route_id','tod']).sum()
    df.reset_index(inplace=True)
    df['source'] = fname.split('.xlsx')[0]

    model = df

    # Join observed data

    df = pd.read_csv(r'C:\Users\Brice\soundcast-summary\daysim\data\transit_boardings_2014.csv')
    df.index = df['PSRC_Rte_ID']
    df.drop([u'Unnamed: 0','PSRC_Rte_ID','SignRt'],axis=1,inplace=True)

    df = pd.DataFrame(df.stack())
    df.reset_index(inplace=True)
    df.rename(columns={0:'board', 'level_1':'hour','PSRC_Rte_ID':'route_id'}, inplace=True)

    # Convert hour to time of day definition
    df['hour'] = df['hour'].apply(lambda row: row.split('_')[-1])
    tod_df = pd.DataFrame(data=tod_lookup.values(),index=tod_lookup.keys(), columns=['tod'])
    tod_df['hour'] = tod_df.index.astype('str')

    df = pd.merge(df,tod_df,on='hour')
    df.drop('hour', axis=1,inplace=True)

    # Group by tod
    df = df.groupby(['tod','route_id']).sum()
    df['tod'] = df.index.get_level_values(0)
    df['route_id'] = df.index.get_level_values(1)
    df.reset_index(inplace=True, drop=True)

    df = pd.merge(model, df, on=['route_id','tod'], suffixes=['_model','_observed'])
    df.rename(columns={'board_model':'model','board_observed':'observed'}, inplace=True)
    df['source'] = fname.split('.xlsx')[0]
    fname_out = 'transit_boardings.csv'
    
    write_csv(df=df, fname=fname_out)

In [12]:
def old_traffic_counts(net_file, fname):
    counts_df = pd.read_excel(net_file, sheetname='Counts Output')

    counts_df.drop([u'OBJECTID_1', u'Join_Count', u'TARGET_FID', u'OBJECTID', u'SR', u'RID', 
             u'MP', u'ARM', u'Type_', u'Lanes', u'Oneway', u'Dir', u'ID',
             u'HOV_I', u'HOV_J'],
           axis=1, inplace=True)
    
    # Model results
    df = counts_df
    df = df[['vol'+str(i) for i in tod_list]+['NewINode']]
    df = df.set_index(keys='NewINode',drop=True)

    # realign
    df = pd.DataFrame(df.stack())

    df['tod'] = df.index.get_level_values(1)
    df['tod'] = df['tod'].apply(lambda row: row.split('vol')[-1])
    df['NewINode'] = df.index.get_level_values(0)
    df.rename(columns={0:'volume'}, inplace=True)
    df.reset_index(inplace=True,drop=True)
    
    df['source'] = fname.split('.xlsx')[0]
    
    fname_out = 'traffic_counts.csv'
    write_csv(df=df, fname=fname_out)

    # Observed results
    if 'observed' not in pd.read_csv(os.path.join(output_dir,fname_out))['source'].values:
    
        df = counts_df
        for i in xrange(24):
            if i < 10:
                df = df.rename(columns={'Vol_0'+str(i): str(i)})
            df = df.rename(columns={'Vol_'+str(i): str(i)})

        df = df[[str(i) for i in xrange(24)]+['NewINode']]
        df = df.set_index(keys='NewINode',drop=True)

        # realign
        df = pd.DataFrame(df.stack())
        df.reset_index(inplace=True)
        df.rename(columns={'level_1':'hour',0:'volume'}, inplace=True)

        # Load tod lookup
        tod_df = pd.DataFrame(data=tod_lookup.values(),index=tod_lookup.keys(), columns=['tod'])
        tod_df['hour'] = tod_df.index.astype('str')

        df = pd.merge(df,tod_df,on='hour')
        df.drop('hour', axis=1,inplace=True)

        # Group by tod
        df = df.groupby(['tod','NewINode']).sum()
        df['tod'] = df.index.get_level_values(0)
        df['NewINode'] = df.index.get_level_values(1)
        df.reset_index(inplace=True, drop=True)

        df['source'] = 'observed'

        write_csv(df=df, fname=fname_out)

In [31]:
def traffic_counts(net_file, fname):
    # note that for now network_summary_detailed is manually updated with soundcast_counts output
    # for traffic counts - should be automated in future to do this immediately
    
    df = pd.read_excel(net_file, sheetname='Counts Output')
        
    # Get total of model @tveh
    # take min of count value because it represents potentially multiple linkes
    df = pd.DataFrame([df.groupby('@scrn').sum()['@tveh'].values,
                       df.groupby('@scrn').min()['@count'].values,
                       df.groupby('@scrn').min()['ul3'].values,
                       df.groupby('@scrn').first()['i'].values,
                       df.groupby('@scrn').first()['j'].values,]).T
    df.columns = ['model','observed','facility','i','j']
    df['source'] = fname.split('.xlsx')[0]
    
    # Add geography
    geog = pd.read_csv(r'data/midpoint_edges_0.txt')
    df = pd.merge(df,geog[['NewINode','NewJNode','lat','lon','county','LARGE_AREA']],
                  left_on=['i','j'], right_on=['NewINode','NewJNode'], how='left')
    
    write_csv(df=df, fname='traffic_counts.csv')

In [32]:
# df = pd.read_excel(r'C:\Users\Brice\surveys\surveys\net_summaries\soundcast.xlsx', sheetname='Counts Output')
# df = pd.DataFrame([df.groupby('@scrn').sum()['@tveh'].values,
#                        df.groupby('@scrn').min()['@count'].values,
#                        df.groupby('@scrn').min()['ul3'].values,
#                        df.groupby('@scrn').min()['i'].values,
#                        df.groupby('@scrn').min()['j'].values,]).T
# df.columns = ['model','observed','facility','i','j']
# df['source'] = fname.split('.xlsx')[0]

# geog = pd.read_csv(r'data/midpoint_edges_0.txt')
# df = pd.merge(df,geog[['NewINode','NewJNode','lat','lon','County','LARGE_AREA']],
#                   left_on=['i','j'], right_on=['NewINode','NewJNode'], how='left')
# df = pd.merge(df,geog[['NewINode','NewJNode','lat','lon','County','LARGE_AREA']],
#                   left_on=['j','j'], right_on=['NewINode','NewJNode'], how='left')

# df

In [35]:
def net_summary(net_file, fname):
    
    net_summary_df = pd.read_excel(net_file, sheetname='Network Summary')
    net_summary_df.index = net_summary_df['tod']
    df = pd.DataFrame(net_summary_df.stack())
    df.reset_index(inplace=True)
    df.rename(columns={0:'value','level_1':'fieldname'}, inplace=True)

    # Drop the rows with TP_4k column headers
    df.drop(df[df['fieldname'] == 'TP_4k'].index, inplace=True)
    df.drop(df[df['fieldname'] == 'tod'].index, inplace=True)

    # Split the fields by vmt, vht, delay
    df['facility_type'] = df.fieldname.apply(lambda row: row.split('_')[0])
    df['metric'] = df.fieldname.apply(lambda row: row.split('_')[-1])

    df['source'] = fname.split('.xlsx')[0]
    
    write_csv(df=df, fname='net_summary.csv')

In [36]:
main()

processing soundcast.xlsx


In [91]:
df = pd.read_excel(r'C:\Users\Brice\surveys\surveys\net_summaries\soundcast.xlsx', sheetname='Counts Output')
df.head()
# Load count data - WSDOT 2 hour counts intersected with transRefEdges
# count_intersect = pd.read_csv(r'C:\Users\Brice\soundcast-summary\daysim\datadata\bike_count_data.csv')

,i,j,ul3,@tveh,@scrn,@count
0,1080,132294,5,2060.224365,1683,34000
1,3735,183223,5,1186.761475,849,2200
2,3748,35642,5,8401.833984,601,16000
3,3844,106246,0,0.000000,773,1600
4,4024,4515,1,28057.796875,214,60000


In [92]:
df.groupby('@scrn').min()['i'].values

array([176609, 176595, 176370, ...,  13355,   7874,   5146], dtype=int64)

In [98]:
df.groupby(['@scrn','i','j']).min().index.get_level_values(1)

Int64Index([176609, 176691, 176691, 176595, 185481, 176370, 176520, 176151, 176235, 175841, 176014, 175853, 185477, 174478, 175560, 174323, 174492, 172938, 173838, 172197, 172907, 171783, 172213, 171199, 171585, 170742, 171268, 170197, 170627, 169161, 169713, 168071, 168603, 166803, 183449, 163959, 166296, 162765, 164301, 161368, 161913, 159080, 160847, 158430, 158974, 155978, 157025, 155418, 155715, 153195, 153305, 152355, 153093, 152425, 184902, 151957, 152089, 151812, 151997, 151500, 151851, 151807, 151834, 199118, 151795, 151858, 199119, 151823, 198275, 199129, 200029, 139771, 142792, 199124, 200018, 138170, 138206, 199712, 199853, 131657, 185403, 199751, 199843, 131158, 131986, 199764, 199844, 125216, 129027, 199630, 199787, 122271, 124208, 199571, 199726, 183689, 199032, 200152, 200155, 118667, ...], dtype='int64')

In [90]:
new_df = pd.DataFrame([df.groupby('@scrn').sum()['@tveh'].values,
                       df.groupby('@scrn').min()['@count'].values,
                       df.groupby('@scrn').min()['ul3'].values]).T
new_df.columns = ['model','observed','facility']


# df['source'] = fname.split('.xlsx')[0]
df
# # Add geography
# geog = pd.read_csv(r'data/midpoint_edges_0.txt')
# df = pd.merge(df,geog[['NewINode','NewJNode','lat','lon','county','LARGE_AREA']],
#               left_on=['i','j'], right_on=['NewINode','NewJNode'], how='left')

# df

,model,observed,facility,i,j
0,124812.387329,114000,1,176609,176595
1,122773.640625,113000,1,176595,176582
2,124698.054688,123000,1,176370,176417
3,120154.007812,114000,1,176151,176166
4,130799.257812,125000,1,175841,175853
5,124121.937500,112000,1,175853,175841
6,129577.238281,121000,1,174478,174492
7,127514.582031,111000,1,174323,174358
8,145291.000000,128000,1,172938,172907
9,139412.710938,118000,1,172197,172213
